## Innebygginger

I vårt forrige eksempel jobbet vi med høy-dimensjonale bag-of-words vektorer med lengde `vocab_size`, og vi konverterte eksplisitt lav-dimensjonale posisjonsrepresentasjonsvektorer til sparsomme én-hot representasjoner. Denne én-hot representasjonen er ikke minneeffektiv. I tillegg behandles hvert ord uavhengig av hverandre, så én-hot kodede vektorer uttrykker ikke semantiske likheter mellom ord.

I denne enheten skal vi fortsette å utforske **News AG**-datasettet. For å begynne, la oss laste inn dataene og hente noen definisjoner fra forrige enhet.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Hva er en embedding?

Ideen med **embedding** er å representere ord ved hjelp av lavdimensjonale tette vektorer som reflekterer den semantiske betydningen av ordet. Vi skal senere diskutere hvordan man bygger meningsfulle ord-embeddings, men for nå kan vi bare tenke på embeddings som en måte å redusere dimensjonaliteten til en ordvektor. 

En embedding-lag tar altså et ord som input og produserer en utgangsvektor med spesifisert `embedding_size`. På en måte ligner det veldig på et `Dense`-lag, men i stedet for å ta en one-hot kodet vektor som input, kan det ta et ordnummer.

Ved å bruke et embedding-lag som det første laget i nettverket vårt, kan vi gå fra bag-of-words til en **embedding bag**-modell, der vi først konverterer hvert ord i teksten vår til den tilsvarende embedding, og deretter beregner en aggregasjonsfunksjon over alle disse embeddingene, som for eksempel `sum`, `average` eller `max`.  

![Bilde som viser en embedding-klassifiserer for fem sekvensord.](../../../../../translated_images/no/embedding-classifier-example.b77f021a7ee67eee.webp)

Vårt klassifiserings-nevrale nettverk består av følgende lag:

* `TextVectorization`-lag, som tar en streng som input og produserer en tensor av token-numre. Vi vil spesifisere en rimelig vokabularstørrelse `vocab_size` og ignorere mindre brukte ord. Input-formen vil være 1, og utgangsformen vil være $n$, siden vi får $n$ tokens som resultat, hver av dem inneholder numre fra 0 til `vocab_size`.
* `Embedding`-lag, som tar $n$ numre og reduserer hvert nummer til en tett vektor med en gitt lengde (100 i vårt eksempel). Dermed vil input-tensoren med form $n$ bli transformert til en $n\times 100$ tensor. 
* Aggregasjonslag, som tar gjennomsnittet av denne tensoren langs den første aksen, dvs. det vil beregne gjennomsnittet av alle $n$ input-tensorer som tilsvarer forskjellige ord. For å implementere dette laget vil vi bruke et `Lambda`-lag og sende inn funksjonen for å beregne gjennomsnittet. Utgangen vil ha formen 100, og det vil være den numeriske representasjonen av hele input-sekvensen.
* Endelig `Dense` lineær klassifiserer.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


I sammendraget, i kolonnen **output-form**, tilsvarer den første tensor-dimensjonen `None` minibatch-størrelsen, og den andre tilsvarer lengden på token-sekvensen. Alle token-sekvenser i minibatch har forskjellige lengder. Vi skal diskutere hvordan vi håndterer dette i neste seksjon.

La oss nå trene nettverket:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Merk** at vi bygger vektorisering basert på et delsett av dataene. Dette gjøres for å akselerere prosessen, og det kan føre til en situasjon der ikke alle tokenene fra teksten vår er til stede i vokabularet. I så fall vil disse tokenene bli ignorert, noe som kan resultere i litt lavere nøyaktighet. Imidlertid gir et delsett av tekst ofte en god vokabularestimering i virkeligheten.


### Håndtering av variabel sekvenslengde

La oss forstå hvordan trening skjer i minibatcher. I eksempelet ovenfor har inndata-tensoren dimensjon 1, og vi bruker minibatcher med lengde 128, slik at den faktiske størrelsen på tensoren er $128 \times 1$. Imidlertid er antallet tokens i hver setning forskjellig. Hvis vi bruker `TextVectorization`-laget på en enkelt inndata, vil antallet tokens som returneres være forskjellig, avhengig av hvordan teksten er tokenisert:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Men når vi bruker vektoriseringen på flere sekvenser, må den produsere en tensor med rektangulær form, så den fyller ubrukte elementer med PAD-tokenet (som i vårt tilfelle er null):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Her kan vi se innebyggingene:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

## Semantiske embeddinger: Word2Vec

I vårt forrige eksempel lærte embedding-laget å kartlegge ord til vektorrepresentasjoner, men disse representasjonene hadde ingen semantisk betydning. Det hadde vært fint å lære en vektorrepresentasjon der lignende ord eller synonymer tilsvarer vektorer som er nær hverandre i henhold til en eller annen vektordistanse (for eksempel euklidisk distanse).

For å oppnå dette må vi forhåndstrene vår embedding-modell på en stor samling tekst ved hjelp av en teknikk som [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Den er basert på to hovedarkitekturer som brukes for å produsere en distribuert representasjon av ord:

 - **Continuous bag-of-words** (CBoW), der vi trener modellen til å forutsi et ord basert på den omkringliggende konteksten. Gitt ngrammet $(W_{-2},W_{-1},W_0,W_1,W_2)$, er målet for modellen å forutsi $W_0$ ut fra $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** er motsatt av CBoW. Modellen bruker det omkringliggende vinduet av kontekstord for å forutsi det nåværende ordet.

CBoW er raskere, mens skip-gram er tregere, men det gjør en bedre jobb med å representere sjeldne ord.

![Bilde som viser både CBoW- og Skip-Gram-algoritmer for å konvertere ord til vektorer.](../../../../../translated_images/no/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

For å eksperimentere med Word2Vec-embedding forhåndstrent på Google News-datasettet, kan vi bruke **gensim**-biblioteket. Nedenfor finner vi ordene som ligner mest på 'neural'.

> **Merk:** Når du først oppretter ordvektorer, kan nedlastingen ta litt tid!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Vi kan også hente vektorinnlegget fra ordet, som skal brukes i treningen av klassifiseringsmodellen. Innlegget har 300 komponenter, men her viser vi bare de første 20 komponentene av vektoren for tydelighet:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Det som er flott med semantiske innebygginger er at du kan manipulere vektorkodingen basert på semantikk. For eksempel kan vi be om å finne et ord hvis vektorrepresentasjon er så nær som mulig ordene *konge* og *kvinne*, og så langt som mulig fra ordet *mann*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Et eksempel ovenfor bruker litt intern GenSym-magi, men den underliggende logikken er faktisk ganske enkel. En interessant ting med innebygginger er at du kan utføre normale vektoroperasjoner på innebyggingsvektorer, og det vil reflektere operasjoner på ords **betydninger**. Eksempelet ovenfor kan uttrykkes i form av vektoroperasjoner: vi beregner vektoren som tilsvarer **KONGE-MANN+KVINNE** (operasjonene `+` og `-` utføres på vektorrepresentasjoner av tilsvarende ord), og deretter finner vi det nærmeste ordet i ordboken til den vektoren:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Vi måtte legge til små koeffisienter til *man*- og *woman*-vektorene – prøv å fjerne dem for å se hva som skjer.

For å finne den nærmeste vektoren bruker vi TensorFlow-mekanismen til å beregne en vektor av avstander mellom vår vektor og alle vektorene i vokabularet, og deretter finner vi indeksen til det minste ordet ved hjelp av `argmin`.


Mens Word2Vec virker som en flott måte å uttrykke ordsemantikk på, har det mange ulemper, inkludert følgende:

* Både CBoW- og skip-gram-modellene er **prediktive embeddings**, og de tar kun hensyn til lokal kontekst. Word2Vec utnytter ikke global kontekst.
* Word2Vec tar ikke hensyn til ordets **morfologi**, altså det faktum at betydningen av et ord kan avhenge av ulike deler av ordet, som roten.

**FastText** forsøker å overvinne den andre begrensningen og bygger videre på Word2Vec ved å lære vektorrepresentasjoner for hvert ord og tegn-n-grammene som finnes i hvert ord. Verdiene av representasjonene blir deretter gjennomsnittlig til én vektor ved hver treningssteg. Selv om dette legger til mye ekstra beregning under pretrening, gjør det det mulig for ord-embeddings å kode sub-ord-informasjon.

En annen metode, **GloVe**, bruker en annen tilnærming til ord-embeddings, basert på faktorisering av ord-kontekst-matrisen. Først bygger den en stor matrise som teller antall forekomster av ord i ulike kontekster, og deretter forsøker den å representere denne matrisen i lavere dimensjoner på en måte som minimerer rekonstruksjonstap.

Gensim-biblioteket støtter disse ord-embeddingene, og du kan eksperimentere med dem ved å endre modellinnlastingskoden ovenfor.


## Bruke forhåndstrente embeddings i Keras

Vi kan endre eksempelet ovenfor for å forhåndsfylle matrisen i vår embedding-lag med semantiske embeddings, som Word2Vec. Ordforrådene til den forhåndstrente embedding og tekstkorpuset vil sannsynligvis ikke samsvare, så vi må velge ett. Her utforsker vi de to mulige alternativene: å bruke tokenizer-ordforrådet og å bruke ordforrådet fra Word2Vec-embeddings.

### Bruke tokenizer-ordforråd

Når vi bruker tokenizer-ordforrådet, vil noen av ordene fra ordforrådet ha tilsvarende Word2Vec-embeddings, og noen vil mangle. Gitt at vår ordforrådsstørrelse er `vocab_size`, og lengden på Word2Vec embedding-vektoren er `embed_size`, vil embedding-laget bli representert av en vektmatrise med formen `vocab_size`$\times$`embed_size`. Vi vil fylle denne matrisen ved å gå gjennom ordforrådet:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


For ord som ikke finnes i Word2Vec-ordforrådet, kan vi enten la dem være nuller, eller generere en tilfeldig vektor.

Nå kan vi definere et innebyggingslag med forhåndstrente vekter:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Merk**: Legg merke til at vi setter `trainable=False` når vi oppretter `Embedding`, noe som betyr at vi ikke trener opp Embedding-laget på nytt. Dette kan føre til at nøyaktigheten blir litt lavere, men det gjør treningen raskere.

### Bruke embedding-ordforråd

Et problem med den tidligere tilnærmingen er at ordfordrådene som brukes i TextVectorization og Embedding er forskjellige. For å løse dette problemet kan vi bruke en av følgende løsninger:
* Trene Word2Vec-modellen på nytt med vårt ordfordråd.
* Laste inn datasettet vårt med ordfordrådet fra den forhåndstrente Word2Vec-modellen. Ordfordråd som brukes til å laste inn datasettet kan spesifiseres under innlasting.

Den sistnevnte tilnærmingen virker enklere, så la oss implementere den. Først og fremst vil vi opprette et `TextVectorization`-lag med det spesifiserte ordfordrådet, hentet fra Word2Vec-embeddingene:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Biblioteket for gensim ord-innkapslinger inneholder en praktisk funksjon, `get_keras_embeddings`, som automatisk vil opprette det tilsvarende Keras-innkapslingslaget for deg.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


En av grunnene til at vi ikke ser høyere nøyaktighet er fordi noen ord fra datasettet vårt mangler i den forhåndstrente GloVe-ordforrådet, og derfor blir de i hovedsak ignorert. For å løse dette kan vi trene våre egne embeddings basert på datasettet vårt.


## Kontekstuelle embeddinger

En viktig begrensning med tradisjonelle forhåndstrente embedding-representasjoner som Word2Vec er at, selv om de kan fange opp noe av betydningen av et ord, kan de ikke skille mellom ulike betydninger. Dette kan føre til problemer i modeller som bruker dem videre.

For eksempel har ordet 'play' forskjellige betydninger i disse to setningene:
- Jeg dro på et **skuespill** på teateret.
- John vil **leke** med vennene sine.

De forhåndstrente embeddingene vi har snakket om representerer begge betydningene av ordet 'play' i samme embedding. For å overkomme denne begrensningen, må vi bygge embeddinger basert på **språkmodellen**, som er trent på en stor tekstsamling og *vet* hvordan ord kan settes sammen i ulike kontekster. Å diskutere kontekstuelle embeddinger er utenfor rammen for denne opplæringen, men vi vil komme tilbake til dem når vi snakker om språkmodeller i neste enhet.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
